In [11]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from data import CirclesDatasetGenerator
from models.gan_utils import preprocess

In [45]:
dimensions = 10
samples = 100

In [46]:
generator = CirclesDatasetGenerator(size=dimensions, n_circles=samples, random_colors=True, random_sizes=True, modes=20)

In [47]:
train_iter, val_iter, test_iter = preprocess(generator, samples, 10)

In [48]:
batch = next(iter(train_iter))

In [49]:
[i.shape for i in batch]

[torch.Size([10, 10, 10, 3]), torch.Size([10, 1])]

In [50]:
# Is this intended behavior?
next(iter(train_iter))

[tensor([[[[ 255.,  255.,  255.],
           [ 255.,  255.,  255.],
           [   6.,    4.,    5.],
           ...,
           [   5.,    4.,    6.],
           [   5.,    4.,    6.],
           [ 255.,  255.,  255.]],
 
          [[ 255.,  255.,  255.],
           [ 255.,  255.,  255.],
           [ 255.,  255.,  255.],
           ...,
           [   5.,    4.,    6.],
           [   5.,    5.,    4.],
           [   5.,    5.,    4.]],
 
          [[ 255.,  255.,  255.],
           [ 255.,  255.,  255.],
           [   3.,    4.,    4.],
           ...,
           [   6.,    7.,    2.],
           [   5.,    5.,    4.],
           [ 255.,  255.,  255.]],
 
          ...,
 
          [[ 255.,  255.,  255.],
           [ 255.,  255.,  255.],
           [   5.,    6.,    7.],
           ...,
           [   6.,    6.,    6.],
           [   5.,    7.,    2.],
           [   2.,    4.,    6.]],
 
          [[ 255.,  255.,  255.],
           [ 255.,  255.,  255.],
           [ 255.,  255